In [ ]:
from bomberman.Environment import Environnement
from bomberman.states.State import State
import torch
from DQN_Agent import BuffedDQNAgent
import time

from typing import Dict, List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import clear_output
from IPython import display

In [ ]:
class ReplayBuffer:
    """A simple replay buffer without batch."""

    def __init__(self, size: int):
        self.buffer = []

        self.max_size = size

    def store(
        self,
        data_point : Dict[str, State or List[int] or float or bool],
    ):
        self.buffer.append(data_point)
        if len(self.buffer) > self.max_size:
            self.buffer.pop(-1)

    def sample_batch(self) -> Dict[str, State or List[int] or float or bool]:
        return self.buffer[np.random.choice(len(self))]

    def __len__(self) -> int:
        return len(self.buffer)

In [ ]:
class Trainer():
	def __init__(
		self, 
		Agent : BuffedDQNAgent, 
        memory_size: int=10000,
        target_update: int=100,
        epsilon_decay: float = 1e-3,
        max_epsilon: float = 0.2,
        min_epsilon: float = 0.001,
        gamma: float = 0.99
	) -> None:
		self.training_agent = Agent(1)
		self.env_training = Environnement(1)

		self.to_beat_agent = Agent(2)
		self.env_to_beat = Environnement(2)

		self.memory = ReplayBuffer(memory_size)

		self.epsilon = max_epsilon
		self.epsilon_decay = epsilon_decay
		self.min_epsilon = min_epsilon
		self.max_epsilon = max_epsilon

		self.target_update = target_update

		self.gamma = gamma

		self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )


	def reset(self):
		# Player 1
		state_training = self.env_training.reset()

		# Player 2
		state_to_beat = self.env_to_beat.get_state()
		return state_training, state_to_beat
		
	def compute_reward(self, old_state, next_state, game_over):
		winner = next_state.as_tuple()[2]
		
		if game_over: # if game is over and player is not the winner, return -1 else 1
			return -10. if not winner or winner != self.training_agent.player_num else 10.
		
		return -1
	
	def _plot(
		self, 
		frame_idx: int, 
		scores: List[float], 
		losses: List[float], 
		epsilons: List[float],
	):
		"""Plot the training progresses."""
		clear_output(True)
		plt.figure(figsize=(20, 5))
		plt.subplot(131)
		plt.title('frame %s. score: %s' % (frame_idx, np.mean(scores[-10:])))
		plt.plot(scores)
		plt.subplot(132)
		plt.title('loss')
		plt.plot(losses)
		plt.subplot(133)
		plt.title('epsilons')
		plt.plot(epsilons)
		plt.show()

	def train(self, steps=10, log_each=1, skip_frames=4, reset_agent_each=10, max_duration=50):
		state_training, state_to_beat = self.reset()
		current_skip = skip_frames
		
		i = 1

		# Use a game duration to force agent to finish the game fast
		# (and avoid starting agent which are random to never end playing)
		game_duration = 0

		epsilons = []
		losses = []
		scores = []
		score = 0
		update_count = 0

		while i <= steps:
			game_over = False

			data_point = {"obs": state_training} # need [obs, action, reward, next_obs, done]

			while current_skip > 0 and not game_over:
				with torch.no_grad(): # Don't compute grad for game playing
					time.sleep(0.001)
					
					# To_beat_agent turn
					tobeat_action = self.to_beat_agent.get_action(state_to_beat)
					state_to_beat = self.env_to_beat.do_action(tobeat_action)

					# Then the training agent turn
					training_action = self.training_agent.get_action(state_training, epsilon=self.epsilon)
					state_training = self.env_training.do_action(training_action)

					# if it is the first step of the observation
					if current_skip == skip_frames:
						data_point["action"] = training_action 

					current_skip -= 1

					if (state_training.winner is not None) or game_duration >= max_duration:
						game_over = True
						break
			
			current_skip = skip_frames
			game_duration += 1

			data_point["next_obs"] = state_training
			data_point["reward"] = self.compute_reward(data_point["obs"], data_point["next_obs"], game_over)
			data_point["done"] = game_over

			score += data_point["reward"]

			if game_over:
				game_duration = 0
				scores.append(score)
				score = 0
				print("game has been reseted and winner was : ", data_point["next_obs"].as_tuple()[2]) # only for debug TODO remove this line
				state_training, state_to_beat = self.reset()

			self.memory.store(data_point)

			with torch.cuda.amp.autocast():
				if len(self.memory) > 1: # No batch size for the moment
					sample = self.memory.sample_batch()

					loss = self.training_agent.update_model(sample)
					losses.append(loss)

					update_count += 1

					if update_count % self.target_update == 0:
						self.training_agent._targe_hard_update()

					self.epsilon = max(
						self.min_epsilon, self.epsilon - (self.max_epsilon - self.min_epsilon) * self.epsilon_decay
					)

					epsilons.append(self.epsilon)

			if update_count % log_each == 0:
				self._plot(i, scores, losses, epsilons)


			if i % reset_agent_each == 0: # reset the to beat agent to the current trained agent
				self.to_beat_agent.brain.load_state_dict(self.training_agent.brain.state_dict())

			if i % log_each == 0:
				print(f"{i} steps done")
			i += 1 # TODO : only update i when a step of training is done with the memory

In [ ]:
trainer = Trainer(BuffedDQNAgent)

In [ ]:
trainer.train(steps=1000, log_each=20, reset_agent_each=100, max_duration=50, skip_frames=4)